In [ ]:
#!pip install torch==1.0.0 torchvision==0.2.2 -f https://download.pytorch.org/whl/cu90/torch_stable.html
%load_ext autoreload
%autoreload 2

import argparse
import os
import torch
import capsulenet_mki
import numpy as np

# setting the hyper parameters
parser = argparse.ArgumentParser(description="Capsule Network.")

parser.add_argument('--epochs', default=50, type=int)
parser.add_argument('--batch_size', default=200, type=int)
parser.add_argument('--lr', default=0.001, type=float,help="Initial learning rate")
parser.add_argument('--lr_decay', default=0.9, type=float,help="The value multiplied by lr at each epoch. Set a larger value for larger epochs")
parser.add_argument('--lam_recon', default=0.0005 * 784, type=float,help="The coefficient for the loss of decoder")
parser.add_argument('-r', '--routings', default=3, type=int,help="Number of iterations used in routing algorithm. should > 0")  # num_routing should > 0
parser.add_argument('--shift_pixels', default=2, type=int,help="Number of pixels to shift at most in each direction.")
parser.add_argument('--data_dir', default='../wind_data/',help="Directory of data. If no data, use \'--download\' flag to download it")
parser.add_argument('--download', action='store_true',help="Download the required data.")
parser.add_argument('--save_dir', default='./result')
parser.add_argument('-t', '--testing', action='store_true',help="Test the trained model on testing dataset")
parser.add_argument('-w', '--weights', default=None,help="The path of the saved weights. Should be specified when testing")
args = parser.parse_args(args=[])
print(args)

if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

# load direction data
input_size=[4, 5, 5]
train_loader, test_loader = capsulenet_mki.load(input_size ,'../data/',200,1) # last argument is location index (0-3)

# define model
model = capsulenet_mki.CapsuleNet(input_size, classes=6, routings=5)
if torch.cuda.is_available():
  model.cuda()

capsulenet_mki.train(model, train_loader, test_loader, args)
test_loss, test_acc = capsulenet_mki.test(model=model, test_loader=test_loader, args=args)
print('test acc = %.4f, test loss = %.5f' % (test_acc, test_loss))

In [ ]:
import argparse
import os
import torch
import capsulenet_mki
import numpy as np
import matplotlib.pyplot as plt

path = '../data/'
rain_bins = np.array([0,0.1,4,16,32,64,100])

y_arr = np.load(path+'rain_basin_NDJFM_19790101-20190331.npy') # rain data
y_arr = np.digitize(y_arr[:,0],rain_bins)-1

plt.hist(y_arr,bins = np.arange(0,6)-0.5)
print(y_arr.shape)

In [ ]:
test_loss, test_acc = capsulenet.test(model=model, test_loader=test_loader, args=args)
print('test acc = %.4f, test loss = %.5f' % (test_acc, test_loss))

In [ ]:
def caps_loss(y_true, y_pred, x, x_recon, lam_recon):
    L = y_true * torch.clamp(0.9 - y_pred, min=0.) ** 2 + \
        0.5 * (1 - y_true) * torch.clamp(y_pred - 0.1, min=0.) ** 2
    L_margin = L.sum(dim=1).mean()

    L_recon = nn.MSELoss()(x_recon, x)

    return L_margin + lam_recon * L_recon
